In [1]:
import socket
import sys

# Función para registrar un nuevo usuario en la base de datos
def register_user(username, password):
    # Leer la base de datos de usuarios existentes
    usuarios=[]
    with open("data.txt", "r") as database:
        lineas = database.readlines()
        for i in range(len(lineas)):
            u=lineas[i].strip().split("-->")[0]
            usuarios.append(u)

    # Verificar si el nombre de usuario ya existe
    if username not in usuarios:
        # Añadir el nuevo usuario a la base de datos
        with open("data.txt", "a") as database:
            database.write(username + "-->" + password + "\n")
        return True
    else:
        return False

# Función para validar el inicio de sesión de un usuario
def login_user(username, password):
    # Crear un diccionario para almacenar usuarios y contraseñas
    dicc={}
    with open("data.txt", "r") as database:
        lineas = database.readlines()
        for i in range(len(lineas)):
            u,p=lineas[i].strip().split("-->")
            dicc[u] = p

    # Verificar si el usuario y la contraseña coinciden
    if username in dicc:
        return dicc[username]==password
    else:
        return False

# Función para manejar la solicitud de registro de un usuario
def handle_register(connection):
    # Solicitar y recibir el username del cliente
    connection.sendall("Enter username:".encode())
    username = connection.recv(1024).decode()

    # Solicitar y recibir la contraseña del cliente
    connection.sendall("Enter password:".encode())
    password = connection.recv(1024).decode()

    # Realizar el registro del usuario y enviar el resultado al cliente
    if register_user(username, password): #==true
        connection.sendall("Registration successful\n".encode())
        print("Registration successful")
    else: #==false
        connection.sendall("Username already exists. Registration failed.\n".encode())

# Función para manejar la solicitud de inicio de sesión de un usuario
def handle_login(connection):
    # Solicitar y recibir el username del cliente
    connection.sendall("Enter username:".encode())
    username = connection.recv(1024).decode()

    # Solicitar y recibir la contraseña del cliente
    connection.sendall("Enter password:".encode())
    password = connection.recv(1024).decode()

    # Realizar la validación del inicio de sesión y enviar el resultado al cliente
    if login_user(username, password): #==true
        connection.sendall("Login succesful\n".encode())
        print("Login successful")
        active_clients[username] = connection
    else: #==false
        connection.sendall("Login failed. Invalid username/password.\n".encode())
        connection.close()

# Función para manejar la solicitud de cierre de conexión
def handle_exit(connection):
    # Enviar un mensaje de despedida al cliente y cerrar la conexión
    connection.sendall("Goodbye".encode())
    connection.close()

# Creación del socket y configuración inicial
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address = ('localhost', 6780)
sock.bind(server_address)
active_clients = {}
sock.listen()

while True:
    print("Waiting for connections...\n")
    connection, client_address = sock.accept()
    data = connection.recv(1024).decode()
    print("Action type: " + data)

    # Manejar la solicitud del cliente según el comando recibido
    if data.lower() == "register":
        handle_register(connection)
    elif data.lower() == "login":
        handle_login(connection)
    elif data.lower() == "exit":
        handle_exit(connection)
        break
    else:
        connection.sendall("Invalid command".encode())


Waiting for connections...

Action type: Register
Registration successful
Waiting for connections...

Action type: Login
Login successful
Waiting for connections...

Action type: exit
